<a href="https://colab.research.google.com/github/afrina23/Single-Image-Crowd-Counting-with-MCNN/blob/master/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting to the google drive for data.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


necessary imports and functions and the model itself

In [0]:
import os
import cv2
import time

import torch
import torch.nn as nn
from torch.autograd import Variable
import random
import pandas as pd

import numpy as np
import sys

#from src.crowd_count import CrowdCounter
#from src import network
#from src.data_loader import ImageDataLoader
#from src.timer import Timer
#from src import utils
#from src.evaluate_model import evaluate_model

try:
    from termcolor import cprint
except ImportError:
    cprint = None

try:
    from pycrayon import CrayonClient
except ImportError:
    CrayonClient = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)


def save_results(input_img, gt_data, density_map, output_dir, fname='results.png'):
    input_img = input_img[0][0]
    gt_data = 255 * gt_data / np.max(gt_data)
    density_map = 255 * density_map / np.max(density_map)
    gt_data = gt_data[0][0]
    density_map = density_map[0][0]
    if density_map.shape[1] != input_img.shape[1]:
        density_map = cv2.resize(density_map, (input_img.shape[1], input_img.shape[0]))
        gt_data = cv2.resize(gt_data, (input_img.shape[1], input_img.shape[0]))
    result_img = np.hstack((input_img, gt_data, density_map))
    cv2.imwrite(os.path.join(output_dir, fname), result_img)


def save_density_map(density_map, output_dir, fname='results.png'):
    density_map = 255 * density_map / np.max(density_map)
    density_map = density_map[0][0]
    cv2.imwrite(os.path.join(output_dir, fname), density_map)


def display_results(input_img, gt_data, density_map):
    input_img = input_img[0][0]
    gt_data = 255 * gt_data / np.max(gt_data)
    density_map = 255 * density_map / np.max(density_map)
    gt_data = gt_data[0][0]
    density_map = density_map[0][0]
    if density_map.shape[1] != input_img.shape[1]:
        input_img = cv2.resize(input_img, (density_map.shape[1], density_map.shape[0]))
    result_img = np.hstack((input_img, gt_data, density_map))
    result_img = result_img.astype(np.uint8, copy=False)
    cv2.imshow('Result', result_img)
    cv2.waitKey(0)




class Timer(object):
    def __init__(self):
        self.tot_time = 0.
        self.calls = 0
        self.start_time = 0.
        self.diff = 0.
        self.average_time = 0.

    def tic(self):
        # using time.time instead of time.clock because time time.clock
        # does not normalize for multithreading
        self.start_time = time.time()

    def toc(self, average=True):
        self.diff = time.time() - self.start_time
        self.tot_time += self.diff
        self.calls += 1
        self.average_time = self.tot_time / self.calls
        if average:
            return self.average_time
        else:
            return self.diff


class ImageDataLoader():
    def __init__(self, data_path, gt_path, shuffle=False, gt_downsample=False, pre_load=False):
        # pre_load: if true, all training and validation images are loaded into CPU RAM for faster processing.
        #          This avoids frequent file reads. Use this only for small datasets.
        self.data_path = data_path
        self.gt_path = gt_path
        self.gt_downsample = gt_downsample
        self.pre_load = pre_load
        #print(os.listdir(data_path))
        
        self.data_files = [filename for filename in os.listdir(data_path) \
                           if os.path.isfile(os.path.join(data_path, filename))]
        self.data_files.sort()
        #print("size of data file ", self.data_files)
        self.shuffle = shuffle
        if shuffle:
            random.seed(2468)
        self.num_samples = len(self.data_files)
        self.blob_list = {}
        self.id_list = range(0, self.num_samples)
        if self.pre_load:
            print
            'Pre-loading the data. This may take a while...'
            idx = 0
            for fname in self.data_files:

                img = cv2.imread(os.path.join(self.data_path, fname), 0)
                img = img.astype(np.float32, copy=False)
                ht = img.shape[0]
                wd = img.shape[1]
                ht_1 = int(ht / 4) * 4
                wd_1 = int(wd / 4) * 4
                img = cv2.resize(img, (wd_1, ht_1))
                img = img.reshape((1, 1, img.shape[0], img.shape[1]))
                den = pd.read_csv(os.path.join(self.gt_path, os.path.splitext(fname)[0] + '.csv'), sep=',',
                                  header=None).as_matrix()
                den = den.astype(np.float32, copy=False)
                #print(wd_1,ht_1)
                if self.gt_downsample:
                    wd_1 = int(wd_1 / 4)
                    ht_1 = int(ht_1 / 4)
                    #print(wd_1,ht_1)
                    den = cv2.resize(den, (wd_1, ht_1))
                    den = den * ((wd * ht) / (wd_1 * ht_1))
                else:
                    den = cv2.resize(den, (wd_1, ht_1))
                    den = den * ((wd * ht) / (wd_1 * ht_1))

                den = den.reshape((1, 1, den.shape[0], den.shape[1]))
                blob = {}
                blob['data'] = img
                blob['gt_density'] = den
                blob['fname'] = fname
                self.blob_list[idx] = blob
                idx = idx + 1
                if idx % 100 == 0:
                    print
                    'Loaded ', idx, '/', self.num_samples, 'files'

            print
            'Completed Loading ', idx, 'files'

    def __iter__(self):
        if self.shuffle:
            if self.pre_load:
                random.shuffle(self.id_list)
            else:
                random.shuffle(self.data_files)
        files = self.data_files
        id_list = self.id_list

        for idx in id_list:
            if self.pre_load:
                blob = self.blob_list[idx]
                blob['idx'] = idx
            else:
                fname = files[idx]
                img = cv2.imread(os.path.join(self.data_path, fname), 0)
                img = img.astype(np.float32, copy=False)
                ht = img.shape[0]
                wd = img.shape[1]
                ht_1 = (ht / 4) * 4
                wd_1 = (wd / 4) * 4
                img = cv2.resize(img, (wd_1, ht_1))
                img = img.reshape((1, 1, img.shape[0], img.shape[1]))
                den = pd.read_csv(os.path.join(self.gt_path, os.path.splitext(fname)[0] + '.csv'), sep=',',
                                  header=None).as_matrix()
                den = den.astype(np.float32, copy=False)
                if self.gt_downsample:
                    wd_1 = wd_1 / 4
                    ht_1 = ht_1 / 4
                    den = cv2.resize(den, (wd_1, ht_1))
                    den = den * ((wd * ht) / (wd_1 * ht_1))
                else:
                    den = cv2.resize(den, (wd_1, ht_1))
                    den = den * ((wd * ht) / (wd_1 * ht_1))

                den = den.reshape((1, 1, den.shape[0], den.shape[1]))
                blob = {}
                blob['data'] = img
                blob['gt_density'] = den
                blob['fname'] = fname

            yield blob

    def get_num_samples(self):
        return self.num_samples


class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, relu=True, same_padding=False, bn=False):
        super(Conv2d, self).__init__()
        padding = int((kernel_size - 1) / 2) if same_padding else 0
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else None
        self.relu = nn.ReLU(inplace=True) if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


class FC(nn.Module):
    def __init__(self, in_features, out_features, relu=True):
        super(FC, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.relu = nn.ReLU(inplace=True) if relu else None

    def forward(self, x):
        x = self.fc(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


def save_net(fname, net):
    import h5py
    h5f = h5py.File(fname, mode='w')
    for k, v in net.state_dict().items():
        h5f.create_dataset(k, data=v.cpu().numpy())


def load_net(fname, net):
    import h5py
    h5f = h5py.File(fname, mode='r')
    for k, v in net.state_dict().items():
        param = torch.from_numpy(np.asarray(h5f[k]))
        v.copy_(param)


def np_to_variable(x, is_cuda=True, is_training=False, dtype=torch.FloatTensor):
    if is_training:
        v = Variable(torch.from_numpy(x).type(dtype))
    else:
        v = Variable(torch.from_numpy(x).type(dtype), requires_grad = False, volatile = True)
    if is_cuda:
        v = v.cuda()
    return v


def set_trainable(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad


def weights_normal_init(model, dev=0.01):
    if isinstance(model, list):
        for m in model:
            weights_normal_init(m, dev)
    else:
        for m in model.modules():
            if isinstance(m, nn.Conv2d):
                #print torch.sum(m.weight)
                m.weight.data.normal_(0.0, dev)
                if m.bias is not None:
                    m.bias.data.fill_(0.0)
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0.0, dev)


class MCNN(nn.Module):
    '''
    Multi-column CNN
        -Implementation of Single Image Crowd Counting via Multi-column CNN (Zhang et al.)
    '''

    def __init__(self, bn=False):
        super(MCNN, self).__init__()

        self.branch1 = nn.Sequential(Conv2d(1, 16, 9, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(16, 32, 7, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(32, 16, 7, same_padding=True, bn=bn),
                                     Conv2d(16, 8, 7, same_padding=True, bn=bn))

        self.branch2 = nn.Sequential(Conv2d(1, 20, 7, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(20, 40, 5, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(40, 20, 5, same_padding=True, bn=bn),
                                     Conv2d(20, 10, 5, same_padding=True, bn=bn))

        self.branch3 = nn.Sequential(Conv2d(1, 24, 5, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(24, 48, 3, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(48, 24, 3, same_padding=True, bn=bn),
                                     Conv2d(24, 12, 3, same_padding=True, bn=bn))

        self.fuse = nn.Sequential(Conv2d(30, 1, 1, same_padding=True, bn=bn))

    def forward(self, im_data):
        x1 = self.branch1(im_data)
        x2 = self.branch2(im_data)
        x3 = self.branch3(im_data)
        x = torch.cat((x1, x2, x3), 1)
        x = self.fuse(x)

        return x






class CrowdCounter(nn.Module):
    def __init__(self):
        super(CrowdCounter, self).__init__()
        self.DME = MCNN()
        self.loss_fn = nn.MSELoss()

    @property
    def loss(self):
        return self.loss_mse

    def forward(self, im_data, gt_data=None):
        im_data = np_to_variable(im_data, is_cuda=True, is_training=self.training)
        density_map = self.DME(im_data)

        if self.training:
            gt_data = np_to_variable(gt_data, is_cuda=True, is_training=self.training)
            self.loss_mse = self.build_loss(density_map, gt_data)

        return density_map

    def build_loss(self, density_map, gt_data):
        loss = self.loss_fn(density_map, gt_data)
        return loss



def evaluate_model(trained_model, data_loader):
    net = CrowdCounter()
    load_net(trained_model, net)
    net.cuda()
    net.eval()
    mae = 0.0
    mse = 0.0
    for blob in data_loader:
        im_data = blob['data']
        gt_data = blob['gt_density']
        density_map = net(im_data, gt_data)
        density_map = density_map.data.cpu().numpy()
        gt_count = np.sum(gt_data)
        et_count = np.sum(density_map)
        mae += abs(gt_count-et_count)
        mse += ((gt_count-et_count)*(gt_count-et_count))
    #print(mae)
    #print(data_loader.get_num_samples())    
    mae = mae/data_loader.get_num_samples()
    mse = np.sqrt(mse/data_loader.get_num_samples())
    return mae,mse




Fine tuning with dataset B or ucf_cc_50

In [0]:
method = 'mcnn_pretrain_A_to_B'
dataset_name = 'ucfc'
output_dir = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_pretrained_last_two_layer_models_A_ucfc/'

train_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/ucfc_patches_9/train'
train_gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/ucfc_patches_9/train_den'
val_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/ucfc_patches_9/val'
val_gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/ucfc_patches_9/val_den'

#train_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/shanghaitech_part_B_patches_9/train'
#train_gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/shanghaitech_part_B_patches_9/train_den'
#val_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/shanghaitech_part_B_patches_9/val'
#val_gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/formatted_trainval/shanghaitech_part_B_patches_9/val_den'


model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_models/mcnn_shtechA_1942.h5'

# training configuration
start_step = 0
end_step = 1000
lr = 0.00001
momentum = 0.9
disp_interval = 500
log_interval = 250

# Tensorboard  config
use_tensorboard = False
save_exp_name = method + '_' + dataset_name + '_' + 'v1'
remove_all_log = False  # remove all historical experiments in TensorBoard
exp_name = None  # the previous experiment name in TensorBoard

# ------------
rand_seed = 64678
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)

# load net
net = CrowdCounter()
trained_model = os.path.join(model_path)
load_net(trained_model, net)

for param in net.parameters():
    param.requires_grad = False

net.DME.branch1[5] = Conv2d(16, 8, 7, same_padding=True, bn=False)

net.DME.branch2[5] = Conv2d(20, 10, 5, same_padding=True, bn=False)

net.DME.branch3[5] = Conv2d(24, 12, 3, same_padding=True, bn=False)

net.DME.fuse = nn.Sequential(Conv2d(30, 1, 1, same_padding=True, bn=False))    
net.cuda()
net.train()


print(net)

total_params = sum(p.numel() for p in net.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in net.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')


params = list(net.parameters())
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# tensorboad
use_tensorboard = use_tensorboard and CrayonClient is not None
if use_tensorboard:
    cc = CrayonClient(hostname='127.0.0.1')
    if remove_all_log:
        cc.remove_all_experiments()
    if exp_name is None:
        exp_name = save_exp_name
        exp = cc.create_experiment(exp_name)
    else:
        exp = cc.open_experiment(exp_name)

# training
train_loss = 0
step_cnt = 0
re_cnt = False
t = Timer()
t.tic()

data_loader = ImageDataLoader(train_path, train_gt_path, shuffle=False, gt_downsample=True, pre_load=True)
data_loader_val = ImageDataLoader(val_path, val_gt_path, shuffle=False, gt_downsample=True, pre_load=True)
best_mae = 10000000000000000
#print(data_loader)
for epoch in range(start_step, end_step + 1):
    step = -1
    train_loss = 0
    print(data_loader.get_num_samples)
    for blob in data_loader:
        step = step + 1
        im_data = blob['data']
        #print(im_data)
        gt_data = blob['gt_density']
        density_map = net(im_data, gt_data)
        loss = net.loss
        #train_loss += loss.data[0]
        #print(type(loss))
        train_loss += loss.item()
        step_cnt += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % disp_interval == 0:
            duration = t.toc(average=False)
            fps = step_cnt / duration
            gt_count = np.sum(gt_data)
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            save_results(im_data, gt_data, density_map, output_dir)
            log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %4.1f, et_cnt: %4.1f' % (epoch,
                                                                                            step, 1. / fps, gt_count,
                                                                                            et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True

        if re_cnt:
            t.tic()
            re_cnt = False

    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method, dataset_name, epoch))
        save_net(save_name, net)
        # calculate error on the validation dataset
        mae, mse = evaluate_model(save_name, data_loader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method, dataset_name, epoch)
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch, mae, mse)
        log_print(log_text, color='green', attrs=['bold'])
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae, best_mse, best_model)
        log_print(log_text, color='green', attrs=['bold'])
        if use_tensorboard:
            exp.add_scalar_value('MAE', mae, step=epoch)
            exp.add_scalar_value('MSE', mse, step=epoch)
            exp.add_scalar_value('train_loss', train_loss / data_loader.get_num_samples(), step=epoch)





CrowdCounter(
  (DME): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (5): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(1, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(k

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:132: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


<bound method ImageDataLoader.get_num_samples of <__main__.ImageDataLoader object at 0x7f878b0fc2b0>>
epoch:    0, step    0, Time: 155.3709s, gt_cnt: 115.3, et_cnt:  0.1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:255: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


EPOCH: 0, MAE: 33.4, MSE: 40.6
BEST MAE: 33.4, BEST MSE: 40.6, BEST MODEL: mcnn_pretrain_A_to_B_ucfc_0.h5
<bound method ImageDataLoader.get_num_samples of <__main__.ImageDataLoader object at 0x7f878b0fc2b0>>
epoch:    1, step    0, Time: 0.0051s, gt_cnt: 115.3, et_cnt:  0.7
<bound method ImageDataLoader.get_num_samples of <__main__.ImageDataLoader object at 0x7f878b0fc2b0>>
epoch:    2, step    0, Time: 0.0025s, gt_cnt: 115.3, et_cnt: 12.9
EPOCH: 2, MAE: 13.2, MSE: 16.2
BEST MAE: 13.2, BEST MSE: 16.2, BEST MODEL: mcnn_pretrain_A_to_B_ucfc_2.h5
<bound method ImageDataLoader.get_num_samples of <__main__.ImageDataLoader object at 0x7f878b0fc2b0>>
epoch:    3, step    0, Time: 0.0017s, gt_cnt: 115.3, et_cnt: 62.7
<bound method ImageDataLoader.get_num_samples of <__main__.ImageDataLoader object at 0x7f878b0fc2b0>>
epoch:    4, step    0, Time: 0.0012s, gt_cnt: 115.3, et_cnt: 92.8
EPOCH: 4, MAE: 14.5, MSE: 19.7
BEST MAE: 13.2, BEST MSE: 16.2, BEST MODEL: mcnn_pretrain_A_to_B_ucfc_2.h5
<bound

testing on dataset shanghaitech part_B or ucfc_cc_50

In [0]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
vis = False
save_output = True

#data_path =  '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/original/shanghaitech/part_B_final/test_data/images/'
#gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/original/shanghaitech/part_B_final/test_data/ground_truth_csv/'
#model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_models/mcnn_shtechA_1942.h5'
#model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_pretrained_last_two_layer_models_A_B/mcnn_pretrain_A_to_B_shtechB_1000.h5'

output_dir = '/content/gdrive/My Drive/project_folder/output/pretrained_last_2_layer_ucfc/'

data_path =  '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/original/UCFC/test_data/images/'
gt_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/data/original/UCFC/test_data/ground_truth_csv/'
#model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_pretrained_models_last_layer/mcnn_pretrain_A_to_B_shtechB_730.h5'
#model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_pretrained_models_last_layer/mcnn_pretrain_A_to_B_shtechB_290.h5'
model_path = '/content/gdrive/My Drive/project_folder/crowdcount-mcnn/saved_pretrained_last_two_layer_models_A_ucfc/mcnn_pretrain_A_to_B_ucfc_2.h5'

#output_dir = '/content/gdrive/My Drive/project_folder/output/pretrained_A_last_layer_test_data_ucfc/'

model_name = os.path.basename(model_path).split('.')[0]
file_results = os.path.join(output_dir,'results_' + model_name + '_.txt')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
output_dir = os.path.join(output_dir, 'density_maps_' + model_name)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)


net = CrowdCounter()
      
trained_model = os.path.join(model_path)
load_net(trained_model, net)



net.cuda()
net.eval()
mae = 0.0
mse = 0.0

#load test data
data_loader = ImageDataLoader(data_path, gt_path, shuffle=False, gt_downsample=True, pre_load=True)
print(net)
for blob in data_loader:                        
    im_data = blob['data']
    gt_data = blob['gt_density']
    density_map = net(im_data, gt_data)
    
    density_map = density_map.data.cpu().numpy()
    gt_count = np.sum(gt_data)
    et_count = np.sum(density_map)
    mae += abs(gt_count-et_count)
    mse += ((gt_count-et_count)*(gt_count-et_count))
    if vis:
        display_results(im_data, gt_data, density_map)
    if save_output:
        save_density_map(density_map, output_dir, 'output_' + blob['fname'].split('.')[0] + '.png')
        
mae = mae/data_loader.get_num_samples()
mse = np.sqrt(mse/data_loader.get_num_samples())
print('\nMAE: %0.2f, MSE: %0.2f' % (mae,mse))

f = open(file_results, 'w') 
f.write('MAE: %0.2f, MSE: %0.2f' % (mae,mse))
f.close()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:132: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


CrowdCounter(
  (DME): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (5): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(1, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(k

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:255: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



MAE: 767.91, MSE: 970.23
